<a href="https://colab.research.google.com/github/LauraMQuiros/ML.Month/blob/main/MLmonth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.dummy import DummyRegressor

data = pd.read_csv("train.csv")
data.head()

,id,title,city,postalCode,latitude,longitude,areaSqm,firstSeenAt,lastSeenAt,isRoomActive,...,living,pets,smokingInside,matchAge,matchGender,matchCapacity,matchLanguages,matchStatus,coverImageUrl,rent
0,0,West-Varkenoordseweg,Rotterdam,3074HN,51.896601,4.514993,14,2019-07-14 11:25:46.511000+00:00,2019-07-26 22:18:23.142000+00:00,True,...,None,No,No,16 years - 99 years,Not important,1 person,Not important,Not important,https://resources.kamernet.nl/image/913b4b03-5...,500
1,3,Ruiterakker,Assen,9407BG,53.013494,6.561012,16,2019-07-14 11:25:46.988000+00:00,2019-07-18 22:00:31.174000+00:00,False,...,None,No,Yes,18 years - 32 years,Female,1 person,Not important,"Student, Working student",https://resources.kamernet.nl/image/84e95365-6...,290
2,8,Brusselseweg,Maastricht,6217GX,50.860841,5.671673,16,2019-07-14 11:25:47.814000+00:00,2019-08-10 00:14:27.130000+00:00,True,...,None,No,No,16 years - 40 years,Male,4 persons,Dutch English,Student,https://resources.kamernet.nl/image/6e625591-d...,425
3,10,Donkerslootstraat,Rotterdam,3074WL,51.893195,4.516478,25,2019-07-14 11:25:48.140000+00:00,2019-07-16 06:05:32.183000+00:00,False,...,None,No,No,21 years - 99 years,Not important,4 persons,Dutch English Spanish French Italian German Po...,"Student, Working student, Working, Looking for...",https://resources.kamernet.nl/image/ea3aea77-0...,600
4,12,Vorselenburgstraat,Alphen aan den Rijn,2405XJ,52.122335,4.661434,10,2019-07-14 11:25:48.465000+00:00,2019-08-01 00:02:40.516000+00:00,True,...,None,No,Yes,22 years - 40 years,Not important,1 person,Dutch English,"Student, Working student, Working",https://resources.kamernet.nl/image/d0780298-b...,425


In [3]:
cols = data.columns
num_cols = data._get_numeric_data().columns
categorical_feat = list(set(cols) - set(num_cols))
sorted(categorical_feat)

['city',
 'coverImageUrl',
 'descriptionNonTranslated',
 'descriptionTranslated',
 'energyLabel',
 'firstSeenAt',
 'furnish',
 'gender',
 'internet',
 'isRoomActive',
 'kitchen',
 'lastSeenAt',
 'living',
 'matchAge',
 'matchCapacity',
 'matchGender',
 'matchLanguages',
 'matchStatus',
 'pets',
 'postalCode',
 'postedAgo',
 'propertyType',
 'rawAvailability',
 'rentDetail',
 'roommates',
 'shower',
 'smokingInside',
 'title',
 'toilet']

In [4]:
ohe_data = pd.get_dummies(
    data[categorical_feat],
    drop_first=True
)

ohe_data.head()

,furnish_Uncarpeted,furnish_Unfurnished,coverImageUrl_https://resources.kamernet.nl/image/0000a5cb-a26b-4409-9cf4-65a64bfbdb80,coverImageUrl_https://resources.kamernet.nl/image/000182e1-9a86-433b-bc92-67bf4f34d3fd,coverImageUrl_https://resources.kamernet.nl/image/00018af5-d72f-4e65-bbae-d8de5b583478,coverImageUrl_https://resources.kamernet.nl/image/00021087-08d0-4097-b066-0058088fc051,coverImageUrl_https://resources.kamernet.nl/image/00027b27-a605-44ff-ade9-f9bfbb5a5f09,coverImageUrl_https://resources.kamernet.nl/image/0002f55e-b440-44af-9390-c83ddc8f7e36,coverImageUrl_https://resources.kamernet.nl/image/0006e79c-6516-42c9-8e59-b46541376803,coverImageUrl_https://resources.kamernet.nl/image/000991f1-a1ea-4cf3-a82c-3cb0a86cdfdc,...,propertyType_Room,propertyType_Student residence,propertyType_Studio,energyLabel_B,energyLabel_C,energyLabel_D,energyLabel_E,energyLabel_F,energyLabel_G,energyLabel_Unknown
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [5]:
num_data = data.drop(categorical_feat, axis=1)

In [4]:
prep_data = pd.concat([
    ohe_data,
    num_data
], axis=1)
prep_data.head()

NameError: ignored

In [1]:
features = prep_data.drop(
    "rent", 
    axis=1
)
target = prep_data["rent"]

NameError: ignored

In [10]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size = .2,
    random_state=17112022
)

NameError: ignored